In [ ]:
import keras
import matplotlib.pyplot as plt
from keras.datasets import cifar10,mnist
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping

In [ ]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
NUM_CHANNELS = 1
epochs = 10
batch_size = 128

In [ ]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters=16, kernel_size=(3,3), activation='relu',strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    x = Conv2D(filters=16, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters=10, kernel_size=(1,1),strides=(1,1), padding='valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def FashionMNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def CIFARmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=2)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=128, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters=10, kernel_size=(1,1),strides=(1,1), padding='valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = MNISTmodel(IMAGE_SIZE,NUM_CLASSES,NUM_CHANNELS)

In [ ]:
model.summary()

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

In [ ]:
def reshape_for_grayscale(x_train, x_test, imsize):
    x_train = x_train.reshape(x_train.shape[0], imsize, imsize, 1)
    x_test = x_test.reshape(x_test.shape[0], imsize, imsize, 1)
    return x_train, x_test

In [ ]:
if NUM_CHANNELS == 1:
    x_train, x_test = reshape_for_grayscale(x_train, x_test, IMAGE_SIZE)
    print("Reshaped data.")

In [ ]:
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

In [ ]:
es = EarlyStopping(patience=3)

In [ ]:
history = model.fit( x_train,y_train,
                     batch_size=batch_size,
                     epochs=epochs,
                     validation_data=(x_test,y_test),
                     shuffle=True,callbacks = [es])

In [ ]:
model.save_weights("model0.h5")

In [ ]:
def plot_and_save_loss(history):
    epoch = len(history.history['loss'])
    epoch = range(1,epoch+1)
    val_loss = history.history['val_loss']
    train_loss = history.history['loss']
    plt.style.use('ggplot')
    plt.plot(epoch,val_loss,'--o', label = 'validation loss')
    plt.plot(epoch, train_loss, '--o', label = 'training loss')
    plt.title("Training and validation loss")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    plt.savefig("model'+str(i)+'.png')

In [ ]:
plot_and_save_loss(history)